# Exercice 1 : Initialisation de l'environnement et des structures de données


In [1]:
import gymnasium as gym
import numpy as np
from collections import defaultdict

In [2]:
env = gym.make("Taxi-v3")

# Nombre d'états et d'actions
state_size = env.observation_space.n
action_size = env.action_space.n

# Création de la table de politique (probabilités égales pour chaque action)
policy_table = np.ones((state_size, action_size)) / action_size

# Création de la table de valeurs initialisée à zéro
value_table = np.zeros(state_size)

# Affichage des premières lignes
print("Policy table (5 premiers états):")
print(policy_table[:5])
print("\nValue table (5 premiers états):")
print(value_table[:5])

Policy table (5 premiers états):
[[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]

Value table (5 premiers états):
[0. 0. 0. 0. 0.]


# Exercice 2 : Exploration et collecte d'épisodes


In [3]:
num_random_episodes = 20

print("\nExécution d'un agent aléatoire:")
for episode in range(num_random_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    actions_executed = []
    rewards_obtained = []
    
    while not done:
        action = env.action_space.sample()  # Action aléatoire
        next_state, reward, done, _, _ = env.step(action)
        
        actions_executed.append(action)
        rewards_obtained.append(reward)
        total_reward += reward
        state = next_state
    
    print(f"Épisode {episode + 1}:")
    print(f"Actions: {actions_executed}")
    print(f"Récompenses: {rewards_obtained}")
    print(f"Récompense totale: {total_reward}\n")


Exécution d'un agent aléatoire:
Épisode 1:
Actions: [np.int64(4), np.int64(4), np.int64(2), np.int64(2), np.int64(0), np.int64(3), np.int64(0), np.int64(5), np.int64(1), np.int64(4), np.int64(2), np.int64(5), np.int64(2), np.int64(4), np.int64(4), np.int64(0), np.int64(2), np.int64(4), np.int64(5), np.int64(4), np.int64(5), np.int64(2), np.int64(2), np.int64(2), np.int64(2), np.int64(1), np.int64(3), np.int64(2), np.int64(4), np.int64(2), np.int64(3), np.int64(4), np.int64(2), np.int64(0), np.int64(4), np.int64(3), np.int64(2), np.int64(5), np.int64(4), np.int64(3), np.int64(4), np.int64(1), np.int64(2), np.int64(2), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(2), np.int64(1), np.int64(4), np.int64(4), np.int64(5), np.int64(1), np.int64(4), np.int64(3), np.int64(2), np.int64(2), np.int64(0), np.int64(1), np.int64(1), np.int64(2), np.int64(2), np.int64(1), np.int64(0), np.int64(2), np.int64(2), np.int64(3), np.int64(1), np.int64(4), np.int64(3), np.int64(1), np.int64(0

# Exercice 3 : Mise à jour de la politique avec PPO


In [ ]:
def ppo_update(episodes, gamma=0.99, epsilon=0.2, alpha=0.01, lr=0.01):
    global policy_table, value_table
    
    for _ in range(episodes):
        # Collecter un épisode
        state, _ = env.reset()
        done = False
        episode_data = []
        
        while not done:
            action_probs = policy_table[state]
            action = np.random.choice(np.arange(action_size), p=action_probs)
            next_state, reward, done, _, _ = env.step(action)
            episode_data.append((state, action, reward))
            state = next_state
        
        # Calcul des récompenses cumulées (discounted rewards)
        states, actions, rewards = zip(*episode_data)
        discounted_rewards = []
        R = 0
        
        for r in reversed(rewards):
            R = r + gamma * R
            discounted_rewards.insert(0, R)
        
        # Normalisation des récompenses cumulées
        discounted_rewards = np.array(discounted_rewards)
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
        
        # Mise à jour de la politique et des valeurs
        for i, (state, action, _) in enumerate(episode_data):
            # Calcul de l'avantage
            advantage = discounted_rewards[i] - value_table[state]
            
            # Ratio des politiques
            old_prob = policy_table[state, action]
            new_policy = policy_table.copy()
            new_policy[state] = np.clip(new_policy[state], 1e-10, 1.0 - 1e-10)  # Éviter les divisions par zéro
            new_policy[state] = new_policy[state] / new_policy[state].sum()  # Renormalisation
            new_prob = new_policy[state, action]
            ratio = new_prob / old_prob
            
            # Mise à jour avec clipping
            clipped_ratio = np.clip(ratio, 1 - epsilon, 1 + epsilon)
            policy_loss = -np.minimum(ratio * advantage, clipped_ratio * advantage)
            
            # Mise à jour de la politique
            policy_table[state, action] += lr * policy_loss
            
            # Renormalisation des probabilités
            policy_table[state] = np.clip(policy_table[state], 1e-10, 1.0 - 1e-10)
            policy_table[state] = policy_table[state] / policy_table[state].sum()
            
            # Mise à jour de la valeur
            value_table[state] += alpha * (discounted_rewards[i] - value_table[state])

# Entraînement avec PPO
print("\nDébut de l'entraînement avec PPO...")
ppo_update(episodes=100)  # 100 épisodes d'entraînement
print("Entraînement terminé!")


Début de l'entraînement avec PPO...


# Exercice 4 : Évaluation de l'agent après entraînement


In [ ]:
num_eval_episodes = 20
total_rewards = []

print("\nÉvaluation de l'agent entraîné:")
for ep in range(num_eval_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action_probs = policy_table[state]
        action = np.argmax(action_probs)  # Choix de l'action la plus probable
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = next_state
    
    total_rewards.append(total_reward)
    print(f"Épisode {ep + 1}: Récompense totale = {total_reward}")

# Comparaison des performances
print("\nRésumé des performances:")
print(f"Récompense moyenne avant entraînement (aléatoire): ~{env.reward_range[0]} à {env.reward_range[1]}")
print(f"Récompense moyenne après entraînement: {np.mean(total_rewards):.2f}")

env.close()